<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("OECD_QNA.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,FREQUENCY,location,measure,subject,indexed_at,provider_code,series_code,series_name,series_num
0,1960,1960-01-01,17316.0,annual,QNA,Quarterly National Accounts,A,AUS,CQR,B1_GS1,2024-10-31T09:51:20.215Z,OECD,AUS.B1_GS1.CQR.A,Australia – Gross domestic product – National ...,Master
1,1961,1961-01-01,17477.0,annual,QNA,Quarterly National Accounts,A,AUS,CQR,B1_GS1,2024-10-31T09:51:20.215Z,OECD,AUS.B1_GS1.CQR.A,Australia – Gross domestic product – National ...,Master
2,1962,1962-01-01,18555.0,annual,QNA,Quarterly National Accounts,A,AUS,CQR,B1_GS1,2024-10-31T09:51:20.215Z,OECD,AUS.B1_GS1.CQR.A,Australia – Gross domestic product – National ...,Master
3,1963,1963-01-01,20254.0,annual,QNA,Quarterly National Accounts,A,AUS,CQR,B1_GS1,2024-10-31T09:51:20.215Z,OECD,AUS.B1_GS1.CQR.A,Australia – Gross domestic product – National ...,Master
4,1964,1964-01-01,22242.0,annual,QNA,Quarterly National Accounts,A,AUS,CQR,B1_GS1,2024-10-31T09:51:20.215Z,OECD,AUS.B1_GS1.CQR.A,Australia – Gross domestic product – National ...,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("OECD_QNA.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,SUBJECT,MEASURE,FREQUENCY,Country,Subject,Measure,Frequency
0,annual,OECD,QNA,Quarterly National Accounts,ARG.P3.CQR.A,Argentina – Final consumption expenditure (P3)...,2004,2004-01-01,366000.7,366000.7,ARG,P3,CQR,A,Argentina,Final consumption expenditure,"National currency, current prices, quarterly l...",Annual
1,annual,OECD,QNA,Quarterly National Accounts,ARG.P3.CQR.A,Argentina – Final consumption expenditure (P3)...,2005,2005-01-01,437901.2,437901.2,ARG,P3,CQR,A,Argentina,Final consumption expenditure,"National currency, current prices, quarterly l...",Annual
2,annual,OECD,QNA,Quarterly National Accounts,ARG.P3.CQR.A,Argentina – Final consumption expenditure (P3)...,2006,2006-01-01,541936.7,541936.7,ARG,P3,CQR,A,Argentina,Final consumption expenditure,"National currency, current prices, quarterly l...",Annual
3,annual,OECD,QNA,Quarterly National Accounts,ARG.P3.CQR.A,Argentina – Final consumption expenditure (P3)...,2007,2007-01-01,677404.4,677404.4,ARG,P3,CQR,A,Argentina,Final consumption expenditure,"National currency, current prices, quarterly l...",Annual
4,annual,OECD,QNA,Quarterly National Accounts,ARG.P3.CQR.A,Argentina – Final consumption expenditure (P3)...,2008,2008-01-01,881871.6,881871.6,ARG,P3,CQR,A,Argentina,Final consumption expenditure,"National currency, current prices, quarterly l...",Annual


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True